In [49]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Mount to Drive

In [50]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [51]:
list_of_subdistricts = "/content/sample_data/districs_of_south jakarta.csv"
df_south_jakarta = pd.read_csv(list_of_subdistricts)
df_south_jakarta

,sub-districs
0,Cilandak
1,Jagakarsa
2,Kebayoran Lama
3,Kebayoran Baru
4,Mampang Prapatan
5,Pancoran
6,Pasar Minggu
7,Pesanggrahan
8,Setiabudi
9,Tebet


In [52]:
latitude = []
longitude = []
for sd in df_south_jakarta['sub-districs']:
  address = '{}, South Jakarta'.format(sd)
  geolocator = Nominatim(user_agent="ny_explorer")
  location = geolocator.geocode(address)
  latitude.append(location.latitude)
  longitude.append(location.longitude)
df_south_jakarta['Latitude'] = latitude
df_south_jakarta['Longitude'] = longitude

df_south_jakarta

,sub-districs,Latitude,Longitude
0,Cilandak,-6.283818,106.804863
1,Jagakarsa,-6.330101,106.822237
2,Kebayoran Lama,-6.249128,106.777782
3,Kebayoran Baru,-6.243164,106.799850
4,Mampang Prapatan,-6.250878,106.823021
5,Pancoran,-6.258085,106.842733
6,Pasar Minggu,-6.291950,106.827835
7,Pesanggrahan,-6.255458,106.763112
8,Setiabudi,-6.221706,106.826308
9,Tebet,-6.226016,106.858396


In [53]:
address = 'South Jakarta, Indonesia'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_south_jakarta = folium.Map(location=[latitude, longitude], zoom_start=10)

In [54]:

# add markers to map
for lat, lng, distric, sub_distric in zip(df_south_jakarta['Latitude'], df_south_jakarta['Longitude'], "South Jakarta", df_south_jakarta['sub-districs']):
    label = '{}, {}'.format(sub_distric, distric)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_south_jakarta)  
    
map_south_jakarta

In [55]:
CLIENT_ID = 'PARHPFYPBADPKA43AY2YGQN5L5GBMYQDY2YIWFB0AUTRS3SP' # your Foursquare ID

CLIENT_SECRET = 'B4ZL1LN2V13SMT4BK4VVGAXPXRBPD31TT0UHGS1KFWO0GHDT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PARHPFYPBADPKA43AY2YGQN5L5GBMYQDY2YIWFB0AUTRS3SP
CLIENT_SECRET:B4ZL1LN2V13SMT4BK4VVGAXPXRBPD31TT0UHGS1KFWO0GHDT


In [83]:
def getNearbyVenues(names, latitudes, longitudes, radius=1200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 100
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Districs', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [84]:
south_jakarta_venues = getNearbyVenues(names=df_south_jakarta['sub-districs'],
                                   latitudes=df_south_jakarta['Latitude'],
                                   longitudes=df_south_jakarta['Longitude']
                                  )
south_jakarta_venues.to_csv(r'south_jakarta_venues.csv')

Cilandak
Jagakarsa
Kebayoran Lama
Kebayoran Baru
Mampang Prapatan
Pancoran
Pasar Minggu
Pesanggrahan
Setiabudi
Tebet


In [85]:
south_jakarta_venues

,Districs,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cilandak,-6.283818,106.804863,Zap Permanent Hair Removal,-6.283846,106.807332,Health & Beauty Service
1,Cilandak,-6.283818,106.804863,Twin House Noodles & Beyond,-6.278725,106.804894,Noodle House
2,Cilandak,-6.283818,106.804863,Bulaf Cafe,-6.287187,106.801288,Café
3,Cilandak,-6.283818,106.804863,Sophie Authentique,-6.277665,106.801904,French Restaurant
4,Cilandak,-6.283818,106.804863,Apotek Aji Waras,-6.278053,106.806364,Pharmacy
5,Cilandak,-6.283818,106.804863,Ayam Goreng Presto,-6.277884,106.803124,Restaurant
6,Cilandak,-6.283818,106.804863,Tuku,-6.277724,106.801506,Coffee Shop
7,Cilandak,-6.283818,106.804863,Jakarta Coffee House,-6.278047,106.804396,Coffee Shop
8,Cilandak,-6.283818,106.804863,Toodz House,-6.277903,106.801816,Café
9,Cilandak,-6.283818,106.804863,Srsly Coffee,-6.277821,106.805712,Coffee Shop


In [86]:
df_duplicated = south_jakarta_venues[south_jakarta_venues.duplicated(subset=['Venue', 'Venue Latitude', 'Venue Longitude'], keep=False)]
df_duplicated

,Districs,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
161,Kebayoran Lama,-6.249128,106.777782,Mie Aceh Bang Jali,-6.253261,106.773631,Noodle House
532,Pesanggrahan,-6.255458,106.763112,Mie Aceh Bang Jali,-6.253261,106.773631,Noodle House
